In [ ]:
import numpy as np
from numpy import ndarray

**Helter functions**

In [ ]:
def assert_same_shape(output: ndarray,
                      output_grad: ndarray):
    assert output.shape == output_grad.shape, \
    '''
    Two ndarray should have the same shape; instead, first ndarray's shape is {0}
    and second ndarray's shape is {1}.
    '''.format(tuple(output_grad.shape), tuple(output.shape))
    return None

In [ ]:
def assert_dim(t: ndarray,
               dim: ndarray):
    assert len(t.shape) == dim, \
    '''
    Tensor expected to have dimension {0}, instead has dimension {1}
    '''.format(dim, len(t.shape))
    return None

**1D Convolution** : 1 input 1 output

**Padding**

In [ ]:
input_1d = np.array([1,2,3,4,5])
param_1d = np.array([1,1,1])

In [ ]:

def _pad_1d(inp: ndarray,
            num: int) -> ndarray:
    z = np.array([0])
    z = np.repeat(z, num)
    # print(z)
    return np.concatenate([z, inp, z])

In [ ]:
_pad_1d(input_1d, 1)

array([0, 1, 2, 3, 4, 5, 0])

**Forward Pass**

In [ ]:
def conv_1d(inp: ndarray,
            param: ndarray) -> ndarray:

    # assert correct dimensions
    assert_dim(inp, 1)
    assert_dim(param, 1)

    # pad the input
    param_len = param.shape[0]
    param_mid = param_len // 2
    inp_pad = _pad_1d(inp, param_mid)

    # initialize the output
    out = np.zeros(inp.shape)

    # perform the 1d convolution
    for o in range(out.shape[0]):
        for p in range(param_len):
            out[o] += param[p] * inp_pad[o+p]

    # ensure shapes didn't change
    assert_same_shape(inp, out)

    return out

In [ ]:
def conv_1d_sum(inp: ndarray,
                param: ndarray) -> ndarray:
    out = conv_1d(inp, param)
    return np.sum(out)

In [ ]:
conv_1d_sum(input_1d, param_1d)

39.0

**Testing Gradients**

In [ ]:
np.random.seed(190220)
print(np.random.randint(0, input_1d.shape[0]))
print(np.random.randint(0, param_1d.shape[0]))

4
0


In [ ]:
np.random.seed(291831)
print(np.random.randint(0, input_1d.shape[0]))
print(np.random.randint(0, param_1d.shape[0]))

1
2


In [ ]:
input_1d_2 = np.array([1,2,3,4,6])
param_1d = np.array([1,1,1])

In [ ]:
print(conv_1d_sum(input_1d_2, param_1d) - conv_1d_sum(input_1d_2, param_1d))


0.0


In [ ]:
input_1d = np.array([1,2,3,4,5])
param_1d_2 = np.array([2,1,1])

In [ ]:
print(conv_1d_sum(input_1d, param_1d_2) - conv_1d_sum(input_1d, param_1d))


10.0


**Gradients**

In [ ]:
def _param_grad_1d(inp: ndarray,
                   param: ndarray,
                   output_grad: ndarray = None) -> ndarray:

    param_len = param.shape[0]
    param_mid = param_len // 2
    input_pad = _pad_1d(inp, param_mid)

    if output_grad is None:
        output_grad = np.ones_like(inp)
    else:
        assert_same_shape(inp, output_grad)

    # Zero padded 1 dimensional convolution
    param_grad = np.zeros_like(param)
    input_grad = np.zeros_like(inp)

    for o in range(inp.shape[0]):
        for p in range(param.shape[0]):
            param_grad[p] += input_pad[o+p] * output_grad[o]

    assert_same_shape(param_grad, param)

    return param_grad

In [ ]:
# import numpy as np

# # Example input array
# inp = np.array([1, 2, 3, 4, 5])

# # Example parameter (weight) array
# param = np.array([0.2, 0.5, 0.3])

# # Optional output gradient array
# output_grad = np.array([1, 1, 1, 1, 1])


# # Call the function with the example data
# param_grad = _param_grad_1d(inp, param, output_grad)
# print("Parameter Gradient:", param_grad)


In [ ]:
def _input_grad_1d(inp: ndarray,
                   param: ndarray,
                   output_grad: ndarray = None) -> ndarray:

    param_len = param.shape[0]
    param_mid = param_len // 2
    inp_pad = _pad_1d(inp, param_mid)

    if output_grad is None:
        output_grad = np.ones_like(inp)
    else:
        assert_same_shape(inp, output_grad)

    output_pad = _pad_1d(output_grad, param_mid)

    # Zero padded 1 dimensional convolution
    param_grad = np.zeros_like(param)
    input_grad = np.zeros_like(inp)

    for o in range(inp.shape[0]):
        for f in range(param.shape[0]):
            input_grad[o] += output_pad[o+param_len-f-1] * param[f]

    assert_same_shape(param_grad, param)

    return input_grad

In [ ]:
_input_grad_1d(input_1d, param_1d)


array([2, 3, 3, 3, 2])

In [ ]:
_param_grad_1d(input_1d, param_1d)

array([10, 15, 14])

**Batch Size of 2**

Pad

In [ ]:
input_1d_batch = np.array([[0,1,2,3,4,5,6],
                           [1,2,3,4,5,6,7]])

In [ ]:

def _pad_1d(inp: ndarray,
            num: int) -> ndarray:
    z = np.array([0])
    z = np.repeat(z, num)
    return np.concatenate([z, inp, z])

In [ ]:
def _pad_1d_batch(inp: ndarray,
                  num: int) -> ndarray:
    outs = [_pad_1d(obs, num) for obs in inp]
    return np.stack(outs)

In [ ]:
_pad_1d_batch(input_1d_batch, 1)


array([[0, 0, 1, 2, 3, 4, 5, 6, 0],
       [0, 1, 2, 3, 4, 5, 6, 7, 0]])

Forward

In [ ]:

def conv_1d_batch(inp: ndarray,
                  param: ndarray) -> ndarray:

    outs = [conv_1d(obs, param) for obs in inp]
    return np.stack(outs)

In [ ]:
conv_1d_batch(input_1d_batch, param_1d)


array([[ 1.,  3.,  6.,  9., 12., 15., 11.],
       [ 3.,  6.,  9., 12., 15., 18., 13.]])

**Gradient**

In [ ]:
def input_grad_1d_batch(inp: ndarray,
                        param: ndarray) -> ndarray:

    out = conv_1d_batch(inp, param)

    out_grad = np.ones_like(out)

    batch_size = out_grad.shape[0]

    grads = [_input_grad_1d(inp[i], param, out_grad[i]) for i in range(batch_size)]

    return np.stack(grads)

In [ ]:
def param_grad_1d_batch(inp: ndarray,
                        param: ndarray) -> ndarray:

    output_grad = np.ones_like(inp)

    inp_pad = _pad_1d_batch(inp, 1)
    out_pad = _pad_1d_batch(inp, 1)

    param_grad = np.zeros_like(param)

    for i in range(inp.shape[0]):
        for o in range(inp.shape[1]):
            for p in range(param.shape[0]):
                param_grad[p] += inp_pad[i][o+p] * output_grad[i][o]

    return param_grad

**Checking gradients for conv_1d_batch**

In [ ]:

def conv_1d_batch_sum(inp: ndarray,
                      fil: ndarray) -> ndarray:
    out = conv_1d_batch(inp, fil)
    return np.sum(out)

In [ ]:
conv_1d_batch_sum(input_1d_batch, param_1d)

133.0

In [ ]:
print(np.random.randint(0, input_1d_batch.shape[0]))
print(np.random.randint(0, input_1d_batch.shape[1]))

1
5


In [ ]:
input_1d_batch_2 = input_1d_batch.copy()
input_1d_batch_2[0][2] += 1
conv_1d_batch_sum(input_1d_batch_2, param_1d) - conv_1d_batch_sum(input_1d_batch, param_1d)


3.0

In [ ]:
input_grad_1d_batch(input_1d_batch, param_1d)


array([[2, 3, 3, 3, 3, 3, 2],
       [2, 3, 3, 3, 3, 3, 2]])

In [ ]:
print(np.random.randint(0, param_1d.shape[0]))


0


In [ ]:

param_1d_2 = param_1d.copy()
param_1d_2[2] += 1
conv_1d_batch_sum(input_1d_batch, param_1d_2) - conv_1d_batch_sum(input_1d_batch, param_1d)


48.0

In [ ]:
param_grad_1d_batch(input_1d_batch, param_1d)


array([36, 49, 48])

**2D Convolutions**

In [ ]:
imgs_2d_batch = np.random.randn(3, 28, 28)


In [ ]:
param_2d = np.random.randn(3, 3)


**Padding**

In [ ]:
def _pad_2d(inp: ndarray,
            num: int):
    '''
    Input is a 3 dimensional tensor, first dimension batch size
    '''
    outs = [_pad_2d_obs(obs, num) for obs in inp]

    return np.stack(outs)

In [ ]:
def _pad_2d_obs(inp: ndarray,
                num: int):
    '''
    Input is a 2 dimensional, square, 2D Tensor
    '''
    inp_pad = _pad_1d_batch(inp, num)

    other = np.zeros((num, inp.shape[0] + num * 2))

    return np.concatenate([other, inp_pad, other])

In [ ]:
_pad_2d(imgs_2d_batch, 1).shape


(3, 30, 30)

**Compute OUtput**

In [ ]:
def _compute_output_obs_2d(obs: ndarray,
                           param: ndarray):
    '''
    Obs is a 2d square Tensor, so is param
    '''
    param_mid = param.shape[0] // 2

    obs_pad = _pad_2d_obs(obs, param_mid)

    out = np.zeros_like(obs)

    for o_w in range(out.shape[0]):
        for o_h in range(out.shape[1]):
            for p_w in range(param.shape[0]):
                for p_h in range(param.shape[1]):
                    out[o_w][o_h] += param[p_w][p_h] * obs_pad[o_w+p_w][o_h+p_h]
    return out

In [ ]:
def _compute_output_2d(img_batch: ndarray,
                       param: ndarray):

    assert_dim(img_batch, 3)

    outs = [_compute_output_obs_2d(obs, param) for obs in img_batch]

    return np.stack(outs)

In [ ]:
_compute_output_2d(imgs_2d_batch, param_2d).shape


(3, 28, 28)

**param grads**

In [ ]:
def _compute_grads_obs_2d(input_obs: ndarray,
                          output_grad_obs: ndarray,
                          param: ndarray) -> ndarray:
    '''
    input_obs: 2D Tensor representing the input observation
    output_grad_obs: 2D Tensor representing the output gradient
    param: 2D filter
    '''

    param_size = param.shape[0]
    output_obs_pad = _pad_2d_obs(output_grad_obs, param_size // 2)
    input_grad = np.zeros_like(input_obs)

    for i_w in range(input_obs.shape[0]):
        for i_h in range(input_obs.shape[1]):
            for p_w in range(param_size):
                for p_h in range(param_size):
                    input_grad[i_w][i_h] += output_obs_pad[i_w+param_size-p_w-1][i_h+param_size-p_h-1] \
                    * param[p_w][p_h]

    return input_grad

def _compute_grads_2d(inp: ndarray,
                      output_grad: ndarray,
                      param: ndarray) -> ndarray:

    grads = [_compute_grads_obs_2d(inp[i], output_grad[i], param) for i in range(output_grad.shape[0])]

    return np.stack(grads)

def _param_grad_2d(inp: ndarray,
                   output_grad: ndarray,
                   param: ndarray) -> ndarray:

    param_size = param.shape[0]
    inp_pad = _pad_2d(inp, param_size // 2)

    param_grad = np.zeros_like(param)
    img_shape = output_grad.shape[1:]

    for i in range(inp.shape[0]):
        for o_w in range(img_shape[0]):
            for o_h in range(img_shape[1]):
                for p_w in range(param_size):
                    for p_h in range(param_size):
                        param_grad[p_w][p_h] += inp_pad[i][o_w+p_w][o_h+p_h] \
                        * output_grad[i][o_w][o_h]
    return param_grad

In [ ]:
img_grads = _compute_grads_2d(imgs_2d_batch,
                              np.ones_like(imgs_2d_batch),
                              param_2d)

In [ ]:
img_grads.shape


(3, 28, 28)

In [ ]:

param_grad = _param_grad_2d(imgs_2d_batch,
                              np.ones_like(imgs_2d_batch),
                              param_2d)
param_grad.shape

(3, 3)